# Identity, Immutability, and .NET Interactive Notebooks

Michael L Perry

> @michaellperry

Code posted to https://github.com/michaellperry/identity-immutability


## Location-Specific Identity

Let's get an instance of PostgreSQL running in Docker:

```
docker run --name postgres -p5432:5432 -e POSTGRES_PASSWORD=mypw -d postgres
```

Connect to the container using this command.

```
docker exec -it postgres /bin/bash
```

Once connected, you can start the PostgreSQL client.

```
psql -U postgres
```

Postgres has an object called a Sequence. It's entire purpose is to generate increasing numbers. Every time someone asks for a number from the sequence, they are guaranteed to get a number different -- and bigger -- than any number that anyone else has seen.

Create a sequence like this:

```
CREATE SEQUENCE id;
```

Then get the next number like this:

```
SELECT nextval('id');
```

To quit the PostgreSQL client, just type `\q` and press enter.
Then type `exit` and press enter to log out of the Docker container.

You can also do that in a client application using a Postgres driver.
This code will install a driver via a NuGet package and then get the next ID from the sequence.

In [ ]:
#r "nuget: Npgsql, 5.0.7"

using Npgsql;

var connString = "Host=localhost;Username=postgres;Password=mypw;Database=postgres";

var conn = new NpgsqlConnection(connString);
await conn.OpenAsync();

var comm = new NpgsqlCommand("SELECT nextval('id')", conn);
var nextId = await comm.ExecuteScalarAsync();
await comm.DisposeAsync();

await conn.DisposeAsync();

nextId

Installed Packages Npgsql, 5.0.7

2